<a href="https://colab.research.google.com/github/karthik939075/karthik_FMML_LABS_PROJECTS/blob/main/FMML_M1L2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Machine Learning terms and metrics

FMML Module 1, Lab 2

In this lab, we will show a part of the ML pipeline by using the California Housing dataset. There are 20640 samples, each with 8 attributes like income of the block, age of the houses per district etc. The task is to predict the cost of the houses per district. We will use the scikit-learn library to load the data and perform some basic data preprocessing and model training. We will also show how to evaluate the model using some common metrics, split the data into training and testing sets, and use cross-validation to get a better estimate of the model's performance.

In [1]:
import numpy as np
from sklearn import datasets
import matplotlib.pyplot as plt

rng = np.random.default_rng(seed=42)

In [2]:
dataset = datasets.fetch_california_housing()
# Dataset description
print(dataset.DESCR)

.. _california_housing_dataset:

California Housing dataset
--------------------------

**Data Set Characteristics:**

    :Number of Instances: 20640

    :Number of Attributes: 8 numeric, predictive attributes and the target

    :Attribute Information:
        - MedInc        median income in block group
        - HouseAge      median house age in block group
        - AveRooms      average number of rooms per household
        - AveBedrms     average number of bedrooms per household
        - Population    block group population
        - AveOccup      average number of household members
        - Latitude      block group latitude
        - Longitude     block group longitude

    :Missing Attribute Values: None

This dataset was obtained from the StatLib repository.
https://www.dcc.fc.up.pt/~ltorgo/Regression/cal_housing.html

The target variable is the median house value for California districts,
expressed in hundreds of thousands of dollars ($100,000).

This dataset was derived

Given below are the list of target values. These correspond to the house value derived considering all the 8 input features and are continuous values. We should use regression models to predict these values but we will start with a simple classification model for the sake of simplicity. We need to just round off the values to the nearest integer and use a classification model to predict the house value.

In [3]:
print("Orignal target values:", dataset.target)

dataset.target = dataset.target.astype(int)

print("Target values after conversion:", dataset.target)
print("Input variables shape:", dataset.data.shape)
print("Output variables shape:", dataset.target.shape)

Orignal target values: [4.526 3.585 3.521 ... 0.923 0.847 0.894]
Target values after conversion: [4 3 3 ... 0 0 0]
Input variables shape: (20640, 8)
Output variables shape: (20640,)


The simplest model to use for classification is the K-Nearest Neighbors model. We will use this model to predict the house value with a K value of 1. We will also use the accuracy metric to evaluate the model.

In [4]:
def NN1(traindata, trainlabel, query):
    """
    This function takes in the training data, training labels and a query point
    and returns the predicted label for the query point using the nearest neighbour algorithm

    traindata: numpy array of shape (n,d) where n is the number of samples and d is the number of features
    trainlabel: numpy array of shape (n,) where n is the number of samples
    query: numpy array of shape (d,) where d is the number of features

    returns: the predicted label for the query point which is the label of the training data which is closest to the query point
    """
    diff = (
        traindata - query
    )  # find the difference between features. Numpy automatically takes care of the size here
    sq = diff * diff  # square the differences
    dist = sq.sum(1)  # add up the squares
    label = trainlabel[np.argmin(dist)]
    return label


def NN(traindata, trainlabel, testdata):
    """
    This function takes in the training data, training labels and test data
    and returns the predicted labels for the test data using the nearest neighbour algorithm

    traindata: numpy array of shape (n,d) where n is the number of samples and d is the number of features
    trainlabel: numpy array of shape (n,) where n is the number of samples
    testdata: numpy array of shape (m,d) where m is the number of test samples and d is the number of features

    returns: the predicted labels for the test data which is the label of the training data which is closest to each test point
    """
    predlabel = np.array([NN1(traindata, trainlabel, i) for i in testdata])
    return predlabel

We will also define a 'random classifier', which randomly allots labels to each sample

In [5]:
def RandomClassifier(traindata, trainlabel, testdata):
    """
    This function takes in the training data, training labels and test data
    and returns the predicted labels for the test data using the random classifier algorithm

    In reality, we don't need these arguments but we are passing them to keep the function signature consistent with other classifiers

    traindata: numpy array of shape (n,d) where n is the number of samples and d is the number of features
    trainlabel: numpy array of shape (n,) where n is the number of samples
    testdata: numpy array of shape (m,d) where m is the number of test samples and d is the number of features

    returns: the predicted labels for the test data which is a random label from the training data
    """

    classes = np.unique(trainlabel)
    rints = rng.integers(low=0, high=len(classes), size=len(testdata))
    predlabel = classes[rints]
    return predlabel

We need a metric to evaluate the performance of the model. Let us define a metric 'Accuracy' to see how good our learning algorithm is. Accuracy is the ratio of the number of correctly classified samples to the total number of samples. The higher the accuracy, the better the algorithm. We will use the accuracy metric to evaluate and compate the performance of the K-Nearest Neighbors model and the random classifier.

In [6]:
def Accuracy(gtlabel, predlabel):
    """
    This function takes in the ground-truth labels and predicted labels
    and returns the accuracy of the classifier

    gtlabel: numpy array of shape (n,) where n is the number of samples
    predlabel: numpy array of shape (n,) where n is the number of samples

    returns: the accuracy of the classifier which is the number of correct predictions divided by the total number of predictions
    """
    assert len(gtlabel) == len(
        predlabel
    ), "Length of the ground-truth labels and predicted labels should be the same"
    correct = (
        gtlabel == predlabel
    ).sum()  # count the number of times the groundtruth label is equal to the predicted label.
    return correct / len(gtlabel)

Let us make a function to split the dataset with the desired probability. We will use this function to split the dataset into training and testing sets. We will use the training set to train the model and the testing set to evaluate the model.

In [7]:
def split(data, label, percent):
    # generate a random number for each sample
    rnd = rng.random(len(label))
    split1 = rnd < percent
    split2 = rnd >= percent

    split1data = data[split1, :]
    split1label = label[split1]
    split2data = data[split2, :]
    split2label = label[split2]
    return split1data, split1label, split2data, split2label

We will reserve 20% of our dataset as the test set. We will not change this portion throughout our experiments

In [8]:
testdata, testlabel, alltraindata, alltrainlabel = split(
    dataset.data, dataset.target, 20 / 100
)
print("Number of test samples:", len(testlabel))
print("Number of train samples:", len(alltrainlabel))
print("Percent of test data:", len(testlabel) * 100 / len(dataset.target), "%")

Number of test samples: 4144
Number of train samples: 16496
Percent of test data: 20.07751937984496 %


## Experiments with splits

Let us reserve some of our train data as a validation set

In [9]:
traindata, trainlabel, valdata, vallabel = split(
    alltraindata, alltrainlabel, 75 / 100)

What is the accuracy of our classifiers on the train dataset?

In [10]:
trainpred = NN(traindata, trainlabel, traindata)
trainAccuracy = Accuracy(trainlabel, trainpred)
print("Training accuracy using nearest neighbour algorithm:", trainAccuracy*100, "%")

trainpred = RandomClassifier(traindata, trainlabel, traindata)
trainAccuracy = Accuracy(trainlabel, trainpred)
print("Training accuracy using random classifier: ", trainAccuracy*100, "%")

Training accuracy using nearest neighbour algorithm: 100.0 %
Training accuracy using random classifier:  16.4375808538163 %


For nearest neighbour, the train accuracy is always 1. The accuracy of the random classifier is close to 1/(number of classes) which is 0.1666 in our case. This is because the random classifier randomly assigns a label to each sample and the probability of assigning the correct label is 1/(number of classes). Let us predict the labels for our validation set and get the accuracy. This accuracy is a good estimate of the accuracy of our model on unseen data.

In [11]:
valpred = NN(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy using nearest neighbour algorithm:", valAccuracy*100, "%")


valpred = RandomClassifier(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy using random classifier:", valAccuracy*100, "%")

Validation accuracy using nearest neighbour algorithm: 34.10852713178294 %
Validation accuracy using random classifier: 16.884689922480618 %


Validation accuracy of nearest neighbour is considerably less than its train accuracy while the validation accuracy of random classifier is the same. However, the validation accuracy of nearest neighbour is twice that of the random classifier. Now let us try another random split and check the validation accuracy. We will see that the validation accuracy changes with the split. This is because the validation set is small and the accuracy is highly dependent on the samples in the validation set. We can get a better estimate of the accuracy by using cross-validation.

In [12]:
traindata, trainlabel, valdata, vallabel = split(
    alltraindata, alltrainlabel, 75 / 100)
valpred = NN(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy using nearest neighbour algorithm:", valAccuracy*100, "%")

Validation accuracy using nearest neighbour algorithm: 34.048257372654156 %


You can run the above cell multiple times to try with different random splits.
We notice that the accuracy is different for each run, but close together.

Now let us compare it with the accuracy we get on the test dataset.

In [13]:
testpred = NN(alltraindata, alltrainlabel, testdata)
testAccuracy = Accuracy(testlabel, testpred)

print("Test accuracy:", testAccuracy*100, "%")

Test accuracy: 34.91795366795367 %


### Try it out for yourself and answer:
1. How is the accuracy of the validation set affected if we increase the percentage of validation set? What happens when we reduce it?
2. How does the size of the train and validation set affect how well we can predict the accuracy on the test set using the validation set?
3. What do you think is a good percentage to reserve for the validation set so that thest two factors are balanced?

Answer for both nearest neighbour and random classifier. You can note down the values for your experiments and plot a graph using  <a href=https://matplotlib.org/stable/gallery/lines_bars_and_markers/step_demo.html#sphx-glr-gallery-lines-bars-and-markers-step-demo-py>plt.plot<href>. Check also for extreme values for splits, like 99.9% or 0.1%

> Exercise: Try to implement a 3 nearest neighbour classifier and compare the accuracy of the 1 nearest neighbour classifier and the 3 nearest neighbour classifier on the test dataset. You can use the KNeighborsClassifier class from the scikit-learn library to implement the K-Nearest Neighbors model. You can set the number of neighbors using the n_neighbors parameter. You can also use the accuracy_score function from the scikit-learn library to calculate the accuracy of the model.

## Multiple Splits

One way to get more accurate estimates for the test accuracy is by using <b>cross-validation</b>. Here, we will try a simple version, where we do multiple train/val splits and take the average of validation accuracies as the test accuracy estimation. Here is a function for doing this. Note that this function will take a long time to execute. You can reduce the number of splits to make it faster.

In [14]:
def AverageAccuracy(alldata, alllabel, splitpercent, iterations, classifier=NN):
    """
    This function takes in the data, labels, split percentage, number of iterations and classifier function
    and returns the average accuracy of the classifier

    alldata: numpy array of shape (n,d) where n is the number of samples and d is the number of features
    alllabel: numpy array of shape (n,) where n is the number of samples
    splitpercent: float which is the percentage of data to be used for training
    iterations: int which is the number of iterations to run the classifier
    classifier: function which is the classifier function to be used

    returns: the average accuracy of the classifier
    """
    accuracy = 0
    for ii in range(iterations):
        traindata, trainlabel, valdata, vallabel = split(
            alldata, alllabel, splitpercent
        )
        valpred = classifier(traindata, trainlabel, valdata)
        accuracy += Accuracy(vallabel, valpred)
    return accuracy / iterations  # average of all accuracies

In [15]:
avg_acc = AverageAccuracy(alltraindata, alltrainlabel, 75 / 100, 10, classifier=NN)
print("Average validation accuracy:", avg_acc*100, "%")
testpred = NN(alltraindata, alltrainlabel, testdata)

print("Test accuracy:", Accuracy(testlabel, testpred)*100, "%")

Average validation accuracy: 33.58463539517022 %
Test accuracy: 34.91795366795367 %


This is a very simple way of doing cross-validation. There are many well-known algorithms for cross-validation, like k-fold cross-validation, leave-one-out etc. This will be covered in detail in a later module. For more information about cross-validation, check <a href=https://en.wikipedia.org/wiki/Cross-validation_(statistics)>Cross-validatioin (Wikipedia)</a>

### Questions
1. Does averaging the validation accuracy across multiple splits give more consistent results?
2. Does it give more accurate estimate of test accuracy?
3. What is the effect of the number of iterations on the estimate? Do we get a better estimate with higher iterations?
4. Consider the results you got for the previous questions. Can we deal with a very small train dataset or validation dataset by increasing the iterations?

1)Yes, **averaging the validation accuracy across multiple splits** (i.e., using techniques like **k-fold cross-validation**) can give more consistent and reliable results. Here’s why:

### 1. **Reduced Variance**
When you train and validate a model on a single split of the data, the performance can vary depending on how the data is split. If the split happens to include harder examples or imbalanced classes, the accuracy may fluctuate. By averaging the accuracy across multiple splits, you reduce the impact of any single split's characteristics, leading to a more **stable and representative performance** measure.

### 2. **Better Generalization**
Cross-validation ensures that the model is tested on different parts of the dataset, allowing you to see how well it generalizes to unseen data. If you only use one train-test split, the model might overfit that specific partition. Averaging across multiple splits mitigates this risk.

### 3. **Insights into Model Robustness**
When you average the validation accuracies across multiple splits, you also get a sense of how sensitive the model is to the data. If the accuracy is consistent across splits, it indicates the model is robust. If the accuracy varies a lot, it may indicate the model is more sensitive to the specific distribution of data in each split.

### 4. **Cross-Validation Techniques**
- **k-fold Cross-Validation**: The dataset is split into `k` equal parts, the model is trained on `k-1` parts, and the remaining part is used for validation. This process is repeated `k` times, with each fold used as the validation set once. The average accuracy across all `k` runs is reported.
  
    - **Example**: For 5-fold cross-validation, the dataset is split into 5 equal parts. The model is trained 5 times, with a different part used as the validation set each time. The final accuracy is the average of the 5 validation accuracies.

- **Stratified k-fold**: If the dataset is imbalanced (e.g., classes have different numbers of samples), stratified k-fold ensures that each fold has a similar class distribution, improving consistency in the results.

### 5. **How to Implement k-Fold Cross-Validation in PyTorch**
Here's a simple way to implement k-fold cross-validation in PyTorch using `sklearn.model_selection.KFold`:

```python
from sklearn.model_selection import KFold
import torch
import torch.optim as optim
import torch.nn as nn
from torchvision.datasets import MNIST
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, Subset

# Define model, criterion, and optimizer
model = SimpleNN()  # Use your own model
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Load dataset
dataset = MNIST(root='./data', train=True, download=True, transform=transforms.ToTensor())

# k-fold cross-validation
k_folds = 5
kf = KFold(n_splits=k_folds, shuffle=True)

fold_accuracies = []

for fold, (train_idx, val_idx) in enumerate(kf.split(dataset)):
    print(f"Fold {fold+1}/{k_folds}")
    
    # Subset data for this fold
    train_subset = Subset(dataset, train_idx)
    val_subset = Subset(dataset, val_idx)
    
    # Loaders for training and validation
    train_loader = DataLoader(train_subset, batch_size=32, shuffle=True)
    val_loader = DataLoader(val_subset, batch_size=32, shuffle=False)
    
    # Train the model
    for epoch in range(5):  # Train for 5 epochs as an example
        model.train()
        for images, labels in train_loader:
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
    
    # Validate the model
    correct, total = 0, 0
    model.eval()
    with torch.no_grad():
        for images, labels in val_loader:
            outputs = model(images)
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()

    # Accuracy for this fold
    accuracy = 100 * correct / total
    print(f"Validation Accuracy for fold {fold+1}: {accuracy:.2f}%")
    fold_accuracies.append(accuracy)

# Average accuracy across all folds
average_accuracy = sum(fold_accuracies) / k_folds
print(f"Average Validation Accuracy: {average_accuracy:.2f}%")
```

### 6. **Conclusion**
Averaging the validation accuracy across multiple splits using cross-validation techniques helps provide a **more consistent, reliable, and generalized performance estimate**. It helps to mitigate the risk of obtaining overly optimistic or pessimistic results based on a single, potentially non-representative train-test split.

2)### 1. **Better Representation of the Dataset**
In a single train-test split, the model is only trained on a subset of the data and validated/tested on another subset. This means the model might not encounter all the important patterns in the training data or the validation set might not fully represent the diversity of the overall dataset.

- With **k-fold cross-validation**, each fold acts as both training and validation at some point. This ensures that the model gets to see all of the data in both roles (training and validation), providing a more holistic view of its performance.
  
### 2. **Reduction of Randomness**
Single train-test splits are susceptible to random variations in the data. For instance, if the test set contains a higher concentration of outlier or difficult samples, it could lead to lower performance, or vice versa.

- By **averaging across multiple splits**, cross-validation reduces the impact of random variations in the train-test split and gives a performance estimate that's less dependent on the particular division of the data.

### 3. **Generalization**
Since the model is tested on multiple different validation sets in cross-validation, it’s forced to generalize better across various data distributions, making the averaged accuracy closer to what you'd expect on truly unseen data (test set).

- In contrast, a model trained and evaluated on a single split could potentially overfit the training set or not fully generalize well to the test set, leading to **overly optimistic or pessimistic estimates**.

### 4. **Estimating Test Accuracy with Cross-Validation**
When cross-validation gives you an average validation accuracy, that estimate is typically closer to what you would observe on a real, unseen test set because it accounts for more variability in the data. You can expect that:

- The **cross-validated accuracy** will be closer to the performance on unseen data (test accuracy), as it simulates the process of training and testing on different partitions of the data multiple times.
  
- **Single train-test splits** may give you test accuracy estimates that could be overly optimistic or pessimistic based on the randomness of the split.

### 5. **Potential Limitations**
While cross-validation gives a more accurate estimate of test accuracy, it’s important to note that:
- The final model deployed to the test set should be trained on the **entire training dataset**, not just on one of the k-folds.
- There may still be slight differences between cross-validated accuracy and test accuracy because test sets are usually "unseen" and can contain outlier or different data distributions that the training/validation data did not.

### Conclusion
In most cases, **averaging the validation accuracy over multiple splits** via cross-validation provides a more reliable and **accurate estimate of test accuracy** than using a single train-test split. It helps to better simulate how the model would perform on unseen data by training and testing the model multiple times on different subsets of the data.

3)Yes, increasing the number of iterations (or **splits/folds**) in techniques like **k-fold cross-validation** can provide a more accurate and reliable estimate of the test accuracy, but there are diminishing returns beyond a certain point. Let’s explore this in more detail:

### 1. **Effect of Increasing the Number of Iterations (Folds)**

- **Fewer Folds (e.g., 3-fold or 5-fold cross-validation)**:
   - When using a lower number of folds (e.g., 3 or 5), each fold contains a relatively large portion of the dataset, meaning each model is trained on fewer data points and validated on more data points in each iteration.
   - This might give a relatively good estimate of performance, but the model could still miss out on important patterns from not seeing enough diverse training examples in each fold.
   - **Advantage**: Fewer iterations lead to faster training, but at the cost of slightly less accurate estimates.
   
- **More Folds (e.g., 10-fold or 20-fold cross-validation)**:
   - As you increase the number of folds, the training set size in each iteration becomes larger, with smaller validation sets. This gives the model more data to train on in each fold and thus provides more robust and accurate results.
   - Each sample is used more frequently in training, improving the model’s exposure to different parts of the data. In turn, this leads to better generalization and more stable estimates of performance.
   - **Advantage**: More folds result in a more robust estimate of test accuracy because the model is trained on a larger variety of data in different combinations.

### 2. **Better Estimates with Higher Iterations?**

Generally, yes, **higher iterations (folds)** can lead to **better estimates** of test accuracy, but only up to a certain point:

- **More reliable estimates**: Higher iterations reduce the variance in accuracy estimates across different validation sets, leading to a more reliable and accurate performance metric.
  
- **Reducing bias**: Training the model on a larger portion of the dataset in each iteration reduces bias (since the model is trained on more data), and smaller validation sets reduce the pessimism in accuracy estimates.

However, the improvement in estimate accuracy **diminishes after a certain point**.

### 3. **Diminishing Returns with Too Many Iterations**

While higher iterations generally give better estimates, there’s a point of **diminishing returns**:

- **10-fold cross-validation** is a common balance between reliable estimates and computational cost.
- Increasing to **20-fold** or higher might provide only marginal improvements in estimate quality while significantly increasing computational time.
  
For example, if you go from 10-fold to 20-fold cross-validation:
- The difference in accuracy estimates might only improve slightly, say from **0.5% to 1%**, but the computational cost would double.

### 4. **Leave-One-Out Cross-Validation (LOO-CV)**
The extreme case of increasing iterations is **Leave-One-Out Cross-Validation (LOO-CV)**, where each sample is used as a validation set, and all others are used for training. While this gives the most comprehensive use of data, it is:
- **Very computationally expensive**, especially for large datasets.
- **Highly variable**, as each validation set contains only a single data point, making the result more sensitive to outliers.

In practice, this is not recommended unless you're working with a very small dataset.

### 5. **Trade-Off Between Accuracy and Computation**
Higher iterations (folds) do give better estimates, but they come at the cost of increased computation time:
- **For small datasets**: It can be feasible to use more folds (e.g., 10-fold or even leave-one-out).
- **For large datasets**: Using too many iterations might not be practical, and **5-10 folds** are usually sufficient.

### Conclusion

- **More iterations (folds)** generally lead to more accurate and consistent estimates of test accuracy, as they reduce the variance associated with the random train-test split.
- However, after a certain point (e.g., beyond 10-fold cross-validation), the improvements in accuracy estimates become marginal, and the **computational cost increases significantly**.
- Therefore, in most cases, using **5-10 folds** strikes a good balance between **accuracy** and **computational efficiency**.



4)Yes, you can somewhat mitigate the limitations of a **very small train or validation dataset** by increasing the number of iterations (folds) in cross-validation. However, this approach has limitations and trade-offs. Let’s break down how increasing iterations helps and where its effectiveness diminishes:

### 1. **How Increasing Iterations Helps with Small Datasets**

- **Maximizing Data Usage**:
   - In small datasets, a major concern is that the model doesn't have enough data to learn from. By increasing the number of iterations (or folds), you're effectively increasing the amount of data the model trains on in each fold. For example, in **10-fold cross-validation**, the model is trained on 90% of the data and validated on 10%. In contrast, a simple train-test split might only give the model 70-80% of the data for training.
   - **More iterations mean more training data for each fold**, which helps the model generalize better by exposing it to a larger portion of the data.

- **More Reliable Estimates**:
   - With a small dataset, validation accuracy can vary widely depending on how the data is split. Increasing iterations reduces the variability in accuracy estimates by validating on multiple small subsets. This makes the performance estimate more stable and reliable, which is especially useful when dealing with limited data.
  
### 2. **The Limitation of Small Datasets**
Despite increasing iterations, there are intrinsic limitations when dealing with very small datasets:

- **Limited Information in the Data**:
   - Small datasets inherently have less information. Increasing the number of folds doesn’t change the fact that the model is only being trained on a limited amount of data. While cross-validation helps maximize the use of available data, it **can’t generate new information**. So, even with more iterations, there’s only so much the model can learn.

- **Model Complexity**:
   - When working with very small datasets, a complex model may still overfit, even if you increase the number of folds. Cross-validation helps mitigate overfitting by validating on multiple sets, but with too little data, the model might still not generalize well.

- **Diminishing Returns**:
   - As mentioned earlier, increasing iterations provides diminishing returns after a certain point. For instance, moving from 5-fold to 10-fold cross-validation gives a noticeable benefit. However, moving from 10-fold to 20-fold or leave-one-out cross-validation (LOO-CV) might not significantly improve accuracy estimates while drastically increasing computation time.

### 3. **Very Small Training Sets**
- When the training set is extremely small (say 50 images per class), increasing the number of iterations might help you get a slightly better estimate of how well the model is performing. However:
  - The model might still **struggle to generalize** because there isn’t enough diverse data for it to learn from.
  - Even with higher iterations, the model could still **overfit**, particularly with deep learning models or highly complex architectures.

### 4. **Very Small Validation Sets**
- If you have a very small validation set, increasing the number of iterations can reduce the likelihood that the validation set is not representative of the overall dataset. With more iterations:
  - The validation accuracy becomes more reliable because you’re validating on different subsets of the data across different folds.
  - Each data point will eventually be part of the validation set in some fold, allowing for a more **comprehensive evaluation**.

### 5. **Alternative Techniques for Small Datasets**

When dealing with very small datasets, you can try other techniques beyond just increasing iterations to improve model performance and estimates:

- **Data Augmentation**:
   - Artificially increasing the size of the training set by augmenting existing data (e.g., rotating, flipping, cropping images) can help the model learn better and generalize more effectively. This is particularly helpful in image data.
  
- **Transfer Learning**:
   - Pretrained models can be fine-tuned on small datasets. Since the pretrained model has already learned general features from a larger dataset, it can adapt well to your small dataset with fewer training examples.

- **Simpler Models**:
   - Use less complex models with fewer parameters. These models are less likely to overfit on small datasets.

- **Regularization**:
   - Applying techniques like **L2 regularization** or **dropout** can help prevent overfitting when training on small datasets.

### 6. **Conclusion: Can Increasing Iterations Alone Solve the Problem?**
While increasing the number of iterations can help improve the accuracy estimate and better utilize small datasets, it **cannot fully overcome the inherent limitations** of a small dataset:

- It **maximizes data usage** and gives a more stable performance estimate by validating on different subsets of the data multiple times.
- But if the dataset is very small, the model still faces challenges like **overfitting** and **lack of diversity** in the training data, which higher iterations alone cannot solve.

Thus, increasing iterations is helpful but often needs to be combined with **data augmentation**, **simpler models**, or even **transfer learning** to achieve better performance on small datasets.

> Exercise: How does the accuracy of the 3 nearest neighbour classifier change with the number of splits? How is it affected by the split size? Compare the results with the 1 nearest neighbour classifier.

To explore how the accuracy of a 3 nearest neighbor (KNN) classifier changes with the number of splits and how it is affected by split size, we can perform a series of experiments. Here's how to structure the experiments and analyze the results, followed by a comparison with a 1 nearest neighbor classifier.

### Experiment Structure

1. **Dataset Preparation**:
   - Choose a dataset (e.g., the Iris dataset or MNIST).
   - Split the dataset into different sizes (e.g., 60% training, 40% testing).

2. **Define Number of Splits**:
   - Use different numbers of splits (e.g., 5, 10, and 20-fold cross-validation).

3. **KNN Classifier**:
   - Implement both the 3-nearest neighbor (3-NN) classifier and the 1-nearest neighbor (1-NN) classifier.

4. **Measure Accuracy**:
   - For each number of splits and each classifier, calculate the average accuracy over multiple iterations.

### Key Concepts to Consider

1. **Effect of Number of Splits**:
   - Increasing the number of splits typically leads to a more reliable estimate of the classifier's accuracy.
   - With more splits, each training set contains a larger portion of the overall dataset, which can help the KNN classifier by providing more examples to learn from.

2. **Effect of Split Size**:
   - Smaller splits (more folds) can lead to more training instances per fold, potentially improving accuracy.
   - However, smaller test sets may lead to higher variance in accuracy estimates.

### Example Results (Hypothetical)

1. **5-Fold Cross-Validation**:
   - **3-NN Accuracy**: 85%
   - **1-NN Accuracy**: 82%

2. **10-Fold Cross-Validation**:
   - **3-NN Accuracy**: 87%
   - **1-NN Accuracy**: 84%

3. **20-Fold Cross-Validation**:
   - **3-NN Accuracy**: 88%
   - **1-NN Accuracy**: 85%

### Analysis

1. **Accuracy Change with Splits**:
   - The accuracy of both classifiers generally increases with the number of splits.
   - The 3-NN classifier tends to perform better than the 1-NN classifier, as it reduces the impact of noise and outliers that might mislead the decision in 1-NN.

2. **Impact of Split Size**:
   - As the number of splits increases (for instance, from 5 to 20), the accuracy of the 3-NN classifier consistently improves. This is likely because:
     - Each fold has a sufficient amount of training data for the classifier to learn the underlying patterns.
     - The model benefits from the majority vote mechanism of 3-NN, which averages out noise better than 1-NN.
   - Conversely, with fewer splits, the test sets are larger, which might lead to more variability in the estimates.

3. **Comparison with 1-NN**:
   - The 1-NN classifier, while simple and often effective, tends to be more sensitive to noise. This sensitivity might lead to slightly lower accuracy compared to the 3-NN classifier across different splits.
   - The 3-NN classifier benefits from considering multiple neighbors, which helps in cases where the nearest neighbor might be an outlier.

### Conclusion

- **Summary**:
  - Increasing the number of splits generally leads to improved accuracy for both 1-NN and 3-NN classifiers, but 3-NN consistently outperforms 1-NN due to its robustness against noise.
  - Split size affects accuracy as well; smaller splits tend to yield more reliable estimates but can lead to variance if test sizes are too small.

- **Practical Implications**:
  - When using KNN classifiers, especially in cases with potential noise or outliers, opting for a multi-neighbor approach (like 3-NN) is generally advisable.
  - The choice of the number of splits should balance between computational efficiency and the need for reliable accuracy estimates.

### Next Steps
To solidify these observations, consider running the actual experiments on a chosen dataset, plotting the accuracy against the number of splits, and analyzing the resulting patterns to draw further conclusions.